In [1]:
from tensorflow import keras, expand_dims
import numpy as np
from modules import data_feed
model_name = 'VisFin_v6.h5'


In [2]:
def make_new_model(dropout = 0):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(9, kernel_size = (9,18),padding='same', input_shape=(201, 43, 1), activation='elu'))
    # model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Conv2D(9, kernel_size = (9,18), padding='same', activation='elu'))
    # model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.MaxPooling2D())
    model.add(keras.layers.Conv2D(9, kernel_size = (6,12),padding='same', activation='elu'))
    # model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.MaxPooling2D())
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(420, activation='elu'))
    # model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(160, activation='elu'))
    model.add(keras.layers.Dense(42, activation='elu'))
    model.add(keras.layers.Dense(7,activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [3]:
try:
    VisFin = keras.models.load_model(model_name)
    print('Already exists!')
except OSError:
    VisFin = make_new_model(dropout=0.1)
    print('Compiling!')
    VisFin.save(model_name)
    print(f'{model_name} saved!')

Compiling!
VisFin_v6.h5 saved!


In [24]:
# # convert to JS
import tensorflowjs
tensorflowjs.converters.save_keras_model(VisFin, f'js_format/{model_name.split(".")[0]}')